# Calculating Weighted Mean, Standard Deviation, $\chi^2$ and $\chi^2$-probability.

Very small notebook testing if your calculation of the weighted mean, the standard deviation, the $\chi^2$-value and the $\chi^2$-probability are correct. We use as example five measurements of a distance (too a hook!) with uncertainty. You should insert your own code in the cells below. If you can run the entire notebook without any (Assertion)errors, your code has passed!

***

### Author(s): 
- Troels C. Petersen (Niels Bohr Institute)

### Date:    
- 10-11-2025 (latest update)

In [ ]:
import numpy as np
from scipy import stats
from iminuit import Minuit, cost
from matplotlib import pyplot as plt

Below we define the data for the measurements of the heights of the hook (in mm, though that is irrelavant here) and their estimated uncertainties:

In [ ]:
dhook = np.array([17.8, 18.1, 17.7, 17.7])
err_dhook = np.array([0.5, 0.3, 0.5, 0.2])

## Testing the weighted mean calculation.

This is where you should start pasting your code! 

In [ ]:
# Own implementation:
def weighted_mean(value, error) :
    assert(len(value) == len(error))
    wmean = np.sum((value/(error**2)) / np.sum(1.0/(error**2)))
    wmean_err = error = np.sqrt(1.0/np.sum(1.0/np.square(error)))
    return wmean, wmean_err

# Numpy version:
mean_weighted_np = np.average(dhook, weights = 1.0/err_dhook**2)
error_weighted_np = np.sqrt(1.0/np.sum(1.0/np.square(err_dhook)))
mean_weighted, error_weighted = weighted_mean(dhook, err_dhook)

print("Numpy method:   ", mean_weighted, error_weighted)
print("Our own method: ", mean_weighted_np, error_weighted_np)

assert np.isclose(mean_weighted, 17.80982367758186)

Testing the weighted error:

In [ ]:
# See above calculations:
assert np.isclose(error_weighted, 0.15056568396854866)

Testing the $\chi^2$-value:

In [ ]:
# Simple ChiSquare calculation:
chi2 = np.sum((dhook - mean_weighted)**2/err_dhook**2)

# Fit to constant function (more advanced):
def cfn(x, const):
    return np.ones_like(x)*const

c = cost.LeastSquares(np.arange(len(dhook)), dhook, err_dhook, cfn)
mfit = Minuit(c, const = 18.0)
mfit.migrad()
print(f"  Chi2 values:         Sum: {chi2:8.6f}       Fit: {mfit.fval:8.6f}")
print(f"  Fit result and uncertainty: ", mfit.values[0], mfit.errors[0])

err_weighted = mfit.errors[0]
pval = stats.chi2.sf(mfit.fval, len(dhook)-1)

plt.errorbar(np.arange(len(dhook)), dhook, err_dhook, ls = '', capsize = 4)
plt.hlines(mfit.values[0], 0, 3, ls = '--')
plt.show()

assert np.isclose(chi2, 1.2857430730478727)

Testing the $\chi^2$-probability:

In [ ]:
# Insert own code here to calculate the chi2 probability, chi2_prob
chi2_prob = stats.chi2.sf(chi2, len(dhook)-1)

print(chi2_prob)
assert np.isclose(chi2_prob, 0.7325212770446814)

# Learning points:

You should:
1. Know the difference between an unweighted and a weighted mean.
2. Know that a weighted mean is like a Chi2 fit with a single parameter (the mean!).
3. Be able to test (with Chi2 and p-value) if input values of weighted mean are consistent.   